# Minimum Viable Retrieval Based Chatbot
So this is a place where I would like to implement what I have learned in the Natural Language Understanding modules embodied in the previous notebooks.

For each of the intents, I might want to do some NER (Named Entity Recognition) that are at the intent level so Eve understands can extract more than just an intent as to make the chatbot able to give object specific responses.

Datacamp gave me ideas on how ELIZA bot.

## RASA Training
I try to see my training results using RASA's infrastructure first before I train it myself below.

### CLI Commands
    rasa init
    rasa train nlu
    rasa load nlu : (Loads the model and trains it again)

## NLU Training Guidelines and Research (Source: Rasa)
I have to make the examples distinct from each other. 10-15 examples is what is recommended to train a Rasa bot in the pretrained embeddings case. I obviously use a supervised training approach.
Make sure they are:
* High quality
* Distinct from other intents

`pretrained_embeddings_spacy`

Pros:
* Better model performance with less training data required
* Faster training and iteration times

Shortcomings
* Not available for all languages
* Pre-trained word embeddings do not cover specific domain words

`Supervised training` - Need more training examples, however there are many advantages:
* It will pick up domain specific vocab
* You can build assistants in any language that can be tokenized
* It allows you to handle more advanced cases like messages with multiple intents
* It's a flexible way to handle complex data

Recommended: 1000 labelled examples or more

### Components for Supervised Embeddings Pipeline:
* `SpacyNLP`
* `SpacyTokenizer`
* `SpacyFeaturizer` - Used to create new features from the tokens
    * Can use CountVectorsFeaturizer
    * Ngrams makes it more robust against typos
* `RegexFeaturizer` - Uses regex and lookup tables to label entities (add additional features to CRF Entity Extractor model. Use them only when necessary.
* `CRFEntityExtractor` - For NER - Can also use spaCy entity extractor
    * Duckling Entity Extractor (you don't have to train NER model from scratch, this is for dates, etc.)
    * Has to come after tokenizer
* `EntitySynonymMapper`
* `SklearnIntentClassifier` /  `EmbeddingIntentClassifier`
    * Sklearn Intent Classifier uses SVM
    * Embedding one is good for learning domain specific language

Work iteratively! More NLU training data is better.


## My Own Training

In [1]:
import random
# Visualization 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
# Data science
import pandas as pd
print(f"Pandas: {pd.__version__}")
import numpy as np
print(f"Numpy: {np.__version__}")# Data science
import pandas as pd
print(f"Pandas: {pd.__version__}")
import numpy as np
print(f"Numpy: {np.__version__}")

# Cool progress bars
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()  # Enable tracking of execution progress

import re
import collections
import yaml

Pandas: 1.0.5
Numpy: 1.18.5
Pandas: 1.0.5
Numpy: 1.18.5


/opt/anaconda3/envs/chatbot/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/anaconda3/envs/chatbot/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Stories

In [3]:
# Define variables
name = "Greg"
weather = "cloudy"

# Echo bot

# Create templates
bot_template = "EVE : {0}"
user_template = "USER : {0}"

# Define a dictionary with the predefined responses
responses = {
  "what's your name?": "my name is {0}".format(name),
  "what's today's weather?": "the weather is {0}".format(weather),
  "default": "default message"
}

# Define a dictionary containing a list of responses for each message
responses = {
  "what's your name?": [
      "my name is {0}".format(name),
      "they call me {0}".format(name),
      "I go by {0}".format(name)
   ],
  "what's today's weather?": [
      "the weather is {0}".format(weather),
      "it's {0} today".format(weather)
    ],
  "default": ["default message"]
}
responses = {'statement': ['tell me more!', 'why do you think that?', 'how long have you felt this way?', 'I find that extremely interesting', 'can you back that up?', 'oh wow!', ':)'], 'question': ["I don't know :(", 'you tell me!']}

# Return the matching response if there is one, default otherwise
def respond(message):
    # Check if the message is in the responses
    if message in responses:
        # Return the matching message
        bot_message = responses[message]
    else:
        # Return the "default" message
        bot_message = responses["default"]
    return bot_message

# Use random.choice() to choose a matching response
def respond(message):
    if message in responses:
        bot_message = random.choice(responses[message])
    else:
        bot_message = random.choice(responses["default"])
    return bot_message

def respond(message):
    # Check for a question mark
    if message.endswith("?"):
        # Return a random question
        return random.choice(responses["question"])
    # Return a random statement
    return random.choice(responses["statement"])

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Send a message to the bot
send_message("hello")

# Send messages ending in a question mark
send_message("what's today's weather?")

# Send messages which don't end with a question mark
send_message("I love building chatbots")

respond('hello')

USER : hello
EVE : why do you think that?
USER : what's today's weather?
EVE : I don't know :(
USER : I love building chatbots
EVE : oh wow!


'oh wow!'

In [6]:
rules = {'do you think (.*)': ['if {0}? Absolutely.', 'No chance'], 'do you remember (.*)': ['Did you think I would forget {0}', "Why haven't you been able to forget {0}", 'What about {0}', 'Yes .. and?'], 'I want (.*)': ['What would it mean if you got {0}', 'Why do you want {0}', "What's stopping you from getting {0}"], 'if (.*)': ["Do you really think it's likely that {0}", 'Do you wish that {0}', 'What do you think about {0}', 'Really--if {0}']}


# Text processing

# Define replace_pronouns()
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me', 'you', message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my', 'your', message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your', 'my', message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you', 'me', message)

    return message

print(replace_pronouns("my last birthday"))
print(replace_pronouns("when you went to Florida"))
print(replace_pronouns("I had my own castle"))

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern, responses in rules.items():
        # Create a match object
        match = re.search(pattern, message)
        if match is not None:
            # Choose a random response
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response.format(phrase)

# Test match_rule
print(match_rule(rules, "do you remember your last birthday"))

# Send the messages
send_message("do you remember your last birthday")
send_message("do you think humans should be worried about AI")
send_message("I want a robot friend")
send_message("what if you could be anything you wanted")

your last birthday
when me went to florida
i had your own castle
What about your last birthday
USER : do you remember your last birthday
EVE : why do you think that?
USER : do you think humans should be worried about AI
EVE : tell me more!
USER : I want a robot friend
EVE : how long have you felt this way?
USER : what if you could be anything you wanted
EVE : tell me more!
